# CELL0027 
## Introduction to Machine Learning for bioimage analysis

In [ ]:
# clone the repo so we have a local copy

if "google.colab" in str(get_ipython()):
    !git clone -q https://github.com/lowe-lab-ucl/CELL0027.git


We start by loading all of the prerequisite Python libraries

In [ ]:
import torch

import numpy as np 
import numpy.typing as npt
import matplotlib.pyplot as plt

from pathlib import Path
from skimage.util import montage
from tqdm import tqdm


In [ ]:
DATAPATH = Path("./CELL0027/data/data.npz")

In [ ]:
data = np.load(DATAPATH)

In [ ]:
for key in data.keys():
    m = montage(data[key], grid_shape=(2, 25))
    fig, ax = plt.subplots(figsize=(16, 2))
    ax.imshow(m)
    ax.axis(False)
    ax.set_title(key.title())
    plt.show()

## 1. Calculate the intensity histograms for each class

In [ ]:
pixels = data["interphase"][0].ravel()

In [ ]:
pixels

## 2. Determine the threshold value for the dataset

In [ ]:
threshold = ...

## 3. Write a function to segment the images based on a threshold value

In [ ]:
def segment(image: npt.NDArray, threshold: float) -> npt.NDArray:
    # implement your segmentation function here
    pass

In [ ]:
y = segment(data["interphase"][0], threshold)

## 4. Write code to perform edge detection using the pytorch convolutional kernel

We start by initialising a 2D convolutional operator using `pytorch`:

$$
\text{out}(N_i,C_{out_j}) = \text{bias}(C_{out_j}) + \sum_{k=0}^{C_{in}-1} \text{weight}(C_{out_j},k)~\star~\text{input}(N_i,k)
$$

Where the input size is $(N,C_{in},H,W)$ and the output size is $(N,C_{out},H,W)$. The dimensions of the image are $(H,W)$, which is $(80,80)$ for this dataset. $N$ is the batch size, or number of images analysed simultaneously. The $\star$ operator is the valid 2D cross-correlation operator. Since we only have one channel for our input data, and we only want to calculate the convolution with a single filter $C_{in} = C_{out} = 1$.

You can read the documentation for the 2D convolutional operator here:
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

In [ ]:
C_in = 1 
C_out = 1

kernel_size = (3, 3)

conv = torch.nn.Conv2d(
    C_in, 
    C_out, 
    kernel_size,
    bias=False,
    padding="same",
)
conv.requires_grad_ = False

Let's look at the weights before we set anything. Notice that they're random:

In [ ]:
conv.weight

Here we can define a kernel for the convolution operator. Given the parameters above, the kernel needs to be of size $(C_{in}, C_{out}, H, W)$:

In [ ]:
sobel_x = np.array([
    # specify the kernel here
])

In [ ]:
sobel_x.shape

In [ ]:
# check that the weights are the correct size
assert sobel_x.shape == conv.weight.shape

In [ ]:
# we need to manually set the weights of the convolutional operator here
with torch.no_grad():
    print(conv.weight.shape)
    conv.weight = torch.nn.Parameter(torch.from_numpy(sobel_x).float())

We can take some example data and apply the convolution operation:

In [ ]:
x = (data["metaphase"].astype(np.float32)[:, None, :])

In [ ]:
x.shape

In [ ]:
with torch.no_grad():
    x_out = conv(torch.from_numpy(x)).numpy()

In [ ]:
x_out.shape

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
ax[0].imshow(
    montage(np.squeeze(x), grid_shape=(5, 10), rescale_intensity=True)
)
ax[0].axis(False)
ax[0].set_title("Raw image data")
ax[1].imshow(
    montage(np.squeeze(x_out), grid_shape=(5, 10), rescale_intensity=True),
)
ax[1].axis(False)
ax[1].set_title("Convolution with Sobel kernel, $G_x$")
plt.show()

## Train a nework to perform edge detection

The goal is to learn the sobel kernel using machine learning. We have an input image, and the ground truth transformed image:

In [ ]:
y_true = torch.from_numpy(x_out)
x_in = torch.from_numpy(x)

We need to define a loss function that let's us measure the error between our models output and the expected value. We'll use the Mean Squared Error (MSE) loss for now since we're performing a regression task.

You can learn about the standard loss functions here:
https://pytorch.org/docs/stable/nn.html#loss-functions

In [ ]:
loss_fn = torch.nn.MSELoss()

Now we need to define a model. Here's a simple one that just performs the convolution operation:

In [ ]:
class SobelModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(
            C_in,
            C_out,
            kernel_size=kernel_size,
            padding="same",
            bias=False,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv(x)

In [ ]:
model = SobelModel()

We can look at the model definition here:

In [ ]:
model

Now we set up an optimiser that will do the gradient descent to perform the optimisation:

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

Finally, we have the pytorch training loop that updates the model based on the calculated loss:

In [ ]:
n_training_iterations = 10_000

loss_history = []


# we perform n training iterations, updating the weights after every iteration
# the model should converge on a solution
for i in tqdm(range(n_training_iterations), desc="Training model"):
    # zero your gradients for every batch!
    optimizer.zero_grad()

    # make predictions for this batch
    y_hat = model(x_in)

    # compute the loss and its gradients
    loss = loss_fn(y_hat, y_true)
    loss.backward()

    # adjust learning weights
    optimizer.step()

    # store the loss
    loss_history.append(loss.item())

In [ ]:
# let's look at the learned kernel, how does it compare to what we expect?
model.conv.weight